In [1]:
from app.db.qdrant import hybrid_search, search_dense, search_sparse
from app.utils.vectorizer import convert_to_sparse_vector, convert_to_dense_vector

/home/intern/anaconda3/envs/fin_chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Kết nối thành công Qdrant localhost:6333. Danh sách collection: collections=[CollectionDescription(name='hybrid_search'), CollectionDescription(name='stockcode_vectors'), CollectionDescription(name='news_hybrid_search'), CollectionDescription(name='news_vectors'), CollectionDescription(name='stockcode_hybrid_search'), CollectionDescription(name='fiinquant_documents')]
Đang load model từ: BAAI/bge-m3
✅ Embedding Model đã được load thành công!


In [2]:
def get_ids_hybrid(question, top_k=3):
    """
    Lấy danh sách id của các câu hỏi liên quan đến câu hỏi đầu vào.
    """
    # Chuyển đổi câu hỏi thành vector
    dense = convert_to_dense_vector(question)
    sparse = convert_to_sparse_vector(question)
    
    # Tìm kiếm trong cơ sở dữ liệu
    results = hybrid_search(dense, sparse, top_k=top_k, collection_name="stockcode_hybrid_search")
    
    # Trả về danh sách id
    return [result.id for result in results]

In [3]:
def get_ids_sparse(question, top_k=3):
    """
    Lấy danh sách id của các câu hỏi liên quan đến câu hỏi đầu vào.
    """
    # Chuyển đổi câu hỏi thành vector
    sparse = convert_to_sparse_vector(question)
    
    # Tìm kiếm trong cơ sở dữ liệu
    results = search_sparse(sparse, top_k=top_k, collection_name="stockcode_hybrid_search")
    
    # Trả về danh sách id
    return [result.id for result in results]

In [4]:
def get_ids_dense(question, top_k=3):
    """
    Lấy danh sách id của các câu hỏi liên quan đến câu hỏi đầu vào.
    """
    # Chuyển đổi câu hỏi thành vector
    dense = convert_to_dense_vector(question)
    
    # Tìm kiếm trong cơ sở dữ liệu
    results = search_dense(dense, top_k=top_k, collection_name="stockcode_hybrid_search")
    
    # Trả về danh sách id
    return [result.id for result in results]

In [5]:
# dọc file csv questions_with_id.csv

import pandas as pd

df = pd.read_csv("questions_with_id.csv")

In [6]:
df.head()

,id,question
0,9e1614c5-1c11-42f7-ba18-43d032712fbb,Microsoft đã cam kết đầu tư bao nhiêu tỷ USD c...
1,ef142b12-7529-4f92-a3ea-de5098cd8a21,ACV đã ghi nhận lợi nhuận gộp đạt bao nhiêu tr...
2,8e40abd4-6b28-4ed0-ad74-5b1cdbc3ac48,Tình hình tài chính của ngân hàng VPB trong qu...
3,099153c0-af98-499e-8254-f86b2c688205,Chi phí SG&A của PNJ trong nửa đầu năm 2024 đã...
4,4da89781-cb96-42c3-be92-3d37a1f0dc3a,Công ty nào có mã chứng khoán PHR và những thô...


In [7]:
def hybrid_evaluation(top_k=3):
    sum = 0
    for index, row in df.iterrows():
        ids = get_ids_hybrid(row.question, top_k=top_k)
        if ids:
            if (row.id in ids):
                sum += 1

    return sum / len(df)

In [8]:
z = hybrid_evaluation()
print(z)

0.54


In [9]:
def dense_evaluation(top_k=3):
    sum = 0
    for index, row in df.iterrows():
        ids = get_ids_dense(row.question, top_k=top_k)
        if ids:
            if (row.id in ids):
                sum += 1

    return sum / len(df)

In [10]:
z = dense_evaluation()
print(z)

0.52


In [11]:
def sparse_evaluation(top_k=3):
    sum = 0
    for index, row in df.iterrows():
        ids = get_ids_sparse(row.question, top_k=top_k)
        if ids:
            if (row.id in ids):
                sum += 1

    return sum / len(df)

In [12]:
z = sparse_evaluation(5)
print(z)

0.32
